# **Question Answer**

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import torch

In [ ]:
def get_answer_using_bert(question, reference_text):
  # Load fined tuned model for QA
  bert_model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

  # Load vocab as well
  bert_tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

  # Apply bert_tokenizer on input text
  input_ids = bert_tokenizer.encode(question, reference_text)
  input_tokens = bert_tokenizer.convert_ids_to_tokens(input_ids)

  # Search index of first [SEP] token
  sep_location = input_ids.index(bert_tokenizer.sep_token_id)
  first_seg_len, second_seg_len = sep_location + 1, len(input_ids) - (sep_location + 1)
  seg_embedding = [0] * first_seg_len + [1] * second_seg_len

  # Run our examples on models
  model_scores = bert_model(torch.tensor([input_ids]), token_type_ids=torch.tensor([seg_embedding]))
  ans_start_loc, ans_end_loc = torch.argmax(model_scores[0]), torch.argmax(model_scores[1])
  result = ' '.join(input_tokens[ans_start_loc:ans_end_loc + 1])

  # Return final result
  result = result.replace('#', '')
  return result

In [ ]:
question = "Where was the Football League founded?"
reference_text = " In 1888, The Football League was founded in England, becoming the first of many professional football competitions. During the 20th century, several of the various kinds of football grew to become some of the most popular team sports in the world."

In [ ]:
get_answer_using_bert(question, reference_text)

'england'